# Notebook for End to End testing in the MID PSI

###### Last updated 23/07/24

- DishLMC in the loop
- SPFRx in the loop
- TMC/SDP/CSP in the loop
- Receives and plots the visibilities to SDP
- Allow for running offset calibration scans.

**Currently, this notebook can only run on Talon1, due to that board being the one connected to the SPFRx ,RXPU must also be checked out**

This notebook Uses Dish LMC, SPFRx, TMC, SDP and CSP to run an end-to-end scan. It also provides the option to run multiple offset scan to enable calibration, following the [Five Point Calibration Scan Controls notebook](https://gitlab.com/ska-telescope/ska-jupyter-scripting/-/blob/main/notebooks/observing/MID_five_point_calibration_scan_controls.ipynb?ref_type=heads).

## 1 Setup

### 1.1 Environment Setup

Start by importing all the libraries needed for this notebook

In [ ]:
import json
import os
import time
from time import sleep

from IPython.display import clear_output
from tango import Database, DevFailed, DeviceProxy

Until notebook clean up is merged need to use this for waiting for state

In [ ]:
spinner = ["⣾", "⣽", "⣻", "⢿", "⡿", "⣟", "⣯", "⣷"]


def wait_for_state(testDevice: DeviceProxy, desired_state, break_on_error=True) -> None:
    """Poll a tango device until either the given observation state is reached, or it throws an error.
    Arguments:
    testDevice -- Tango Device to check
    desired_state -- The state which to break upon getting (number or state)
    break_on_error -- If set to False, will keeping running when getting an error status.
    """
    spinL = 0
    poll = 1
    while testDevice.obsState != desired_state:
        if spinL < len(spinner) - 1:
            spinL += 1
        else:
            spinL = 0
        sleep(0.5)
        print(
            "\r",
            f"{spinner[spinL]} Poll# {poll}: Current state is {testDevice.obsState.name}, waiting for {desired_state}...",
            end="",
        )
        if testDevice.obsState == 9 and break_on_error:
            break
        poll += 1
    print(f"\nFinished with: {testDevice.obsState.name}")

### 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [ ]:
!kubectl get ns | grep ska-mid-psi

Now, use this to set the namespace the notebook will use.

In [ ]:
psi_namespace = "ci-ska-mid-psi-1379504815-alexschell"
config = "hw_config_psi.yaml"

Now, load in the other variables this notebook will use, along with config files to pass in. These vars should not need to be changed in most use cases.

In [ ]:
TANGO_HOST = f"databaseds-tango-base.{psi_namespace}.svc.cluster.local:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

RECEPTORS = ["SKA001"]

# For running offset scans
SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

# Only target SKA001 for now as has SPFRx connection
TARGET_BOARD = [1]

SERVER = server = "ska-sdp-kafka." + psi_namespace + ".svc.cluster.local:9092"

START_CHANNEL = 0
END_CHANNEL = 14860
START_PORT = 21000

# Config files set up
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/psi/tmc"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"

CBF_CONFIGS = f"{DATA_DIR}/psi/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"

# MCS/Talon setup
COMMON_CONFIG_FOLDER = os.path.join(DATA_DIR, "psi/cbf")
HW_CONFIG_FOLDER = os.path.join(COMMON_CONFIG_FOLDER, "hw_config")
HW_CONFIG_FILE = os.path.join(HW_CONFIG_FOLDER, config)
INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG_FOLDER, "sys_params/initial_system_param.json")
# Check files can be reached.
files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    DISH_CONFIG_FILE,
    RELEASE_RESOURCES_FILE,
    HW_CONFIG_FILE,
]
for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

Along with these files, the hardware config can also be loaded into the pod via kubectl:

In [ ]:
!kubectl cp $HW_CONFIG_FILE $psi_namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

With the namespace set, the useful front ends to monitor the behaviour of the system can be accessed using the following URLs.

In [ ]:
print(f"https://142.73.34.170/{psi_namespace}/signal/display/")
print(f"https://142.73.34.170/{psi_namespace}/taranta/devices")
print(
    f"https://142.73.34.170/{psi_namespace}/taranta/dashboard?id=666cb28b5e5d4f0012197e5f&mode=run"
)

### 1.3 Setup Device Proxies

Now, set the TANGO FQDNs that will be used to set up device proxies:

In [ ]:
# TMC proxies
CENTRAL_NODE_FQDN = "ska_mid/tm_central/central_node"
CSP_MASTER_FQDN = "ska_mid/tm_leaf_node/csp_master"
LEAF_NODE_SUBARRAY_FQDN = "ska_mid/tm_leaf_node/csp_subarray01"
TMC_SUBARRAY_FQDN = "ska_mid/tm_subarray_node/1"

DEPLOYER_FQDN = "mid_csp_cbf/ec/deployer"

# CSP.LMC proxies
CSP_CONTROL_FQDN = "mid-csp/control/0"
CSP_SUBARRAY_FQDN = "mid-csp/subarray/01"

# CBF proxies
CBF_CONTROLLER_FQDN = "mid_csp_cbf/sub_elt/controller"
CBF_SUBARRAY_FQDN = "mid_csp_cbf/sub_elt/subarray_01"

# Dish Leaf Proxies
LEAF_NODE_FQDN = "ska_mid/tm_leaf_node/d0001"
DISH_MANAGER_FQDN = "mid-dish/dish-manager/ska001"

# SDP Proxies
SDP_SUBARRAY_FQDN = "mid-sdp/subarray/01"

# Leaf Node Proxies
CSP_LEAF_NODE_FQDN = "ska_mid/tm_leaf_node/csp_subarray01"
SDP_LEAF_NODE_FQDN = "ska_mid/tm_leaf_node/sdp_subarray01"
CSP_MASTER_LEAF_NODE_FQDN = "ska_mid/tm_leaf_node/csp_master"

Once these have been set, they can be used to start up the devices:

In [ ]:
# TMC Proxies
tmc_central_node = DeviceProxy(CENTRAL_NODE_FQDN)
tmc_csp_master = DeviceProxy(CSP_MASTER_FQDN)
tmc_csp_subarray = DeviceProxy(LEAF_NODE_SUBARRAY_FQDN)
tmc_subarray = DeviceProxy(TMC_SUBARRAY_FQDN)

# CSP Proxies
csp_control = DeviceProxy(CSP_CONTROL_FQDN)
csp_subarray = DeviceProxy(CSP_SUBARRAY_FQDN)

# CBF Proxies
cbf_controller = DeviceProxy(CBF_CONTROLLER_FQDN)
cbf_subarray = DeviceProxy(CBF_SUBARRAY_FQDN)

# Dish Leaf Proxies
dish_leaf_node_ska001 = DeviceProxy(LEAF_NODE_FQDN)

# SDP Proxies
sdp_subarray = DeviceProxy(SDP_SUBARRAY_FQDN)

# Leaf Node Proxies
csp_subarray_leaf_node_dp = DeviceProxy(CSP_LEAF_NODE_FQDN)
sdp_subarray_leaf_node_dp = DeviceProxy(SDP_LEAF_NODE_FQDN)
csp_master_leaf_node_dp = DeviceProxy(CSP_MASTER_LEAF_NODE_FQDN)

# Deployer
deployer_tango = DeviceProxy(DEPLOYER_FQDN)

# print the states of each
devices = [
    tmc_central_node,
    tmc_csp_master,
    tmc_csp_subarray,
    tmc_subarray,
    csp_control,
    csp_subarray,
    cbf_controller,
    cbf_subarray,
    dish_leaf_node_ska001,
    sdp_subarray,
    csp_subarray_leaf_node_dp,
    sdp_subarray_leaf_node_dp,
    csp_master_leaf_node_dp,
    deployer_tango,
]

for device in devices:
    padding = "-" * (40 - len(device.dev_name()))
    print(f"{device.dev_name()}'s state {padding}> {device.state()}")

### 1.4 MCS Deployer Setup and Download Requirements

To use the deployer, turn it on and set the board that will be deployed to (in this case SKA001).

In [ ]:
db = Database()
# Make sure the deployer device is set to ON
deployer_tango.On()
print(deployer_tango.state())

deployer_tango.targetTalons = TARGET_BOARD
print(deployer_tango.targetTalons)
deployer_tango.generate_config_jsons()

Now the actual download step can be run, this will take some time.

In [ ]:
deployer_tango.set_timeout_millis(400000)
try:
    deployer_tango.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer_tango.set_timeout_millis(4500)

Once the downloaded, the TANGO device database can be configured with the new downloads

In [ ]:
deployer_tango.configure_db()

If desired, the devices can be checked to ensure they have been downloaded.

In [ ]:
print(*db.get_device_exported("*").value_string, sep="\n")

## 2 Set Up Devices

### 2.1 Setting up the TMC Central Node

In [ ]:
# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0
sleep(1)
print("\nChecking admin mode after setting to ONLINE (0):")
print(f"  CSP Control: {csp_control.adminMode}")
print(f"  CSP Subarray: {csp_subarray.adminMode}")
print(f"  CBF Controller: {cbf_controller.adminMode}")
print(f"  CBF Subarray: {cbf_subarray.adminMode}")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99
sleep(2)
print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

### 2.2 Load the Dish Vcc Config / Init Sys Params

Next, load in the dish config file to the central node:

NOTE: RUN TWICE

In [ ]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))

sleep(5)
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{tmc_csp_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

Ensure the VCC configuration has been set by running the following:

In [ ]:
if tmc_central_node.isDishVccConfigSet:
    print("Dish VCC config set, good to proceed!")
else:
    print("Dish VCC config not set.")

### 2.3 Turn the Telescope On

Now, turn the telescope itself on (TelescopeOn may complain about VCC config, have to wait for it to resolve):

In [ ]:
print("Running the TelescopeOn command")
tmc_central_node.set_timeout_millis(100000)
tmc_central_node.TelescopeOn()

startup_time = 0
while int(tmc_central_node.telescopeState) != 0:
    print(f"Telescope is starting up, {startup_time} seconds elapsed.")
    sleep(5)
    startup_time += 5
    print("\r", end="")
print(f"telescope has started after {startup_time} seconds.")
print("Verifying the states:")
print(f"  TMC Central Node State: {tmc_central_node.State()}")
print(f"  CSP Control State: {csp_control.State()}")
print(f"  CBF Controller State: {cbf_controller.State()}")
print(f"  TMC Subarray State: {tmc_subarray.State()}")

### 2.4 Assign Resources to the Telescope

Start assigning resources by setting up the JSON file as needed:

In [ ]:
with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS

print(f"\nassign_resources_json file contents: \n{assign_resources_json}")

Now, the command to actually assign the resources can be run, and the TMC should go to idle

In [ ]:
tmc_subarray.AssignResources(json.dumps(assign_resources_json))
wait_for_state(tmc_subarray, 2)

If required, the slew can be set via the dish manager:

In [ ]:
dish_manager = DeviceProxy(DISH_MANAGER_FQDN)
dish_manager.slew([181.0, 31.0])

while dish_manager.achievedpointing[1] != 181.0 or dish_manager.achievedpointing[2] != 31.0:
    time.sleep(1)

print("Done slewing")

## 3 Running the Scan(s)

With everything set up, the scans can now be run:

### 3.1 Configure Scan

Before running the scan configuration, ensure that the SDP vis pod has spun up:

In [ ]:
!kubectl -n $namespace-sdp get pods | grep vis-receive

Now, append the scan config JSON file as needed before uploading it.

In [ ]:
print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

print(f"\nconfigure_scan_json file contents: \n{configure_scan_json}")

print(json.dumps(configure_scan_json))

Send the configuration, and wait for the TMC to go to ready:

In [ ]:
tmc_subarray.Configure(json.dumps(configure_scan_json))
wait_for_state(tmc_subarray, 4)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### 3.2 Running a Non-Offset Scan

Now, the scan itself can be run by sending the command to the TMC:

**Currently this method will get stuck in the configuring stage, use the following cell to get around this**
**Pending SKB-399**

In [ ]:
print("Running the Scan command: subarray obsstate should go to Scanning")

with open(SCAN_FILE, encoding="utf-8") as f:
    scan_json = f.read()

print(f"\nscan_json file contents: \n{scan_json}")

tmc_subarray.Scan(scan_json)

sleep(2)
wait_for_state(cbf_subarray, 5)
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

For the moment, run this instead to get around the configuration issue:

In [ ]:
with open("../../data/psi/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(sdp_scan)
    sdp_subarray_leaf_node_dp.scan(json.dumps(sdp_scan))

sleep(10)
print(sdp_subarray_leaf_node_dp.sdpSubarrayObsState)

with open("../../data/psi/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_subarray_leaf_node_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_subarray_leaf_node_dp.cspSubarrayObsState)

### 3.3 Ending Initial Scan


After allowing the scan to run for a while, end it via the appropriate command:

**As above, this cannot be run until SKB-399 is resolved**

In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready state")

tmc_subarray.EndScan()
wait_for_state(tmc_subarray, 4)
print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState}")

Use this method in the meantime:

In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready state")

sdp_subarray_leaf_node_dp.EndScan()
sleep(2)
csp_subarray_leaf_node_dp.EndScan()
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### 3.4 Running Offset Scans for Multi-Point Calibration (Optional)

**This is currently blocked as reconfiguration causes low-level TANGO memory allocation issues**

Now, loop through the offset scans, using the combos to set the scan json. For each, configure it by passing in JSON, then scan again, letting it run before stopping.

In [ ]:
count = 4
sdp_subarray_leaf_node_dp.EndScan()

for offset in SCAN_COMBOS:
    print(f"setting offset to: {offset}")

    # Configure for this offset scan
    partial_configure_json = {
        "interface": "https://schema.skao.int/ska-tmc-configure/2.2",
        "transaction_id": f"txn-....-0000{count}",
        "scan_id": count,
        "pointing": {"target": {"ca_offset_arcsec": offset[0], "ie_offset_arcsec": offset[1]}},
        "tmc": {"partial_configuration": True},
    }
    print("Partial Config to load:")
    print(json.dumps(partial_configure_json, indent=1))
    print(".......")
    sdp_subarray_leaf_node_dp.Configure(json.dumps(partial_configure_json))
    wait_for_state(cbf_subarray, 4)
    sleep(10)
    # Send the Scan command along with relevant JSON,incrementing the scan ID and transaction ID
    partial_scan_json = {
        "interface": "https://schema.skao.int/ska-tmc-scan/2.1",
        "transaction_id": f"txn-....-0000{count+4}",
        "scan_id": 2 + count,
    }
    sdp_subarray_leaf_node_dp.Scan(json.dumps(partial_scan_json))
    wait_for_state(cbf_subarray, 5)
    # let the scan run for a bit...
    sleep(200)
    sdp_subarray_leaf_node_dp.EndScan()
    wait_for_state(cbf_subarray, 4)

    count += 1
    print("============================")
print("Done offsets!")

## 4 Cleanup

Now that the scan(s) are complete, the namespace and devices can be shut down and cleaned up:

Start by ending any running scans.

**For use when SKB-399 is resolved:**

In [ ]:
print("Running the End command: subarray obsstate should go to Idle state")

tmc_subarray.End()

sleep(2)
print(f"\nTMC Subarray Observation State: {cbf_subarray.obsState}")

For now use:

In [ ]:
print("Running the End command: subarray obsstate should go to Idle state")

sdp_subarray_leaf_node_dp.End()
sleep(2)
csp_subarray_leaf_node_dp.End()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### 4.1 Release Resources

Once the scans are stopped and finished, clear the resources from the appropriate devices.

**For use when SKB-399 is resolved:**

In [ ]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

tmc_subarray.ReleaseAllResources()
while tmc_subarray.obsState != 0:
    sleep(5)
    clear_output(wait=True)
    print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState}")

For now use:

In [ ]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

sdp_subarray_leaf_node_dp.ReleaseAllResources()
sleep(2)
csp_subarray_leaf_node_dp.ReleaseAllResources()

while sdp_subarray_leaf_node_dp.obsState != 0 and csp_subarray_leaf_node_dp.obsState != 0:
    sleep(5)
    clear_output(wait=True)
    print("These should both return to EMPTY")
    print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
    print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### 4.2 Turn the Telescope Off

Finally, send the Off command to the TMC

In [ ]:
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

while tmc_central_node.State() != 1:
    sleep(5)
    clear_output(wait=True)
    print("These should all go to OFF")
    print(f"TMC Central Node State: {tmc_central_node.State()}")
    print(f"CSP Control State: {csp_control.State()}")
    print(f"CBF Controller State: {cbf_controller.State()}")